In [1]:
import tensorflow as tf
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.mlab import PCA

In [2]:
#Step - 1 :
# Read the input file 
# extract the data from the input file
# return the data 

def readfile(string):
    file = open(string)
    lines = file.readlines()
    line = 1
    data2 = []
    for l in lines:
        inputdata = l.split(' ')
        inputdata = inputdata[0:len(inputdata)-1]
        data2.append( [float(x) for x in inputdata])
        line = line+1
    return data2


In [3]:
# Step - 2: 
# perform the PCA for the input data
# Given input data has 90 features

def doPCA(x):
    mean = []
    for i in range(len(x[0])):
        mean.append(np.mean(x[:,i]))
    mean = np.array(mean)
    #mean is calculated
    
    cov_mat = np.cov([x[i,:] for i in range(len(x[0]))])
    #covariance matrix is calculated
    
    eig_val_cov, eig_vec_cov = np.linalg.eig(cov_mat)
    
    for i in range(len(eig_vec_cov)):
        eigvec_cov = eig_vec_cov[:,i].reshape(1,len(x[0])).T
        #print('Eigenvalue {} from covariance matrix: {}'.format(i+1, eig_val_cov[i]))

    eig_pairs = [(np.abs(eig_val_cov[i]), eig_vec_cov[:,i]) for i in range(len(eig_val_cov))]
    #sort the eigen pairs
    eig_pairs.sort(key=lambda x: x[0], reverse=True)
    #for i in eig_pairs:
    #    print(i[0])
    
    tot = sum(eig_val_cov)
    var_exp = [(i / tot)*100 for i in sorted(eig_val_cov, reverse=True)]
    cum_var_exp = np.cumsum(var_exp)
    print(var_exp[0:10])

    res = sum(var_exp[0:10])
    
    #Now reshape the above matrix into another matrix with having 10 PCA features.
    matrix_w = np.hstack((eig_pairs[0][1].reshape(90,1), 
                          eig_pairs[1][1].reshape(90,1),
                          eig_pairs[2][1].reshape(90,1),
                          eig_pairs[3][1].reshape(90,1),
                          eig_pairs[4][1].reshape(90,1),
                          eig_pairs[5][1].reshape(90,1),
                          eig_pairs[6][1].reshape(90,1),
                          eig_pairs[7][1].reshape(90,1),
                          eig_pairs[8][1].reshape(90,1),
                          eig_pairs[9][1].reshape(90,1),
                    ))
    
    print('Max Covariance after dimensionality reduction : ',res)
    return matrix_w

In [4]:
#Step -3 build a network model.

def neural_network_model(data):
    #I have taken 3 hidden layers and 1 output layer 
    # all the three hidden layers use sigmoid activation function 
    # output layer use tanh activation function
    hidden_1_layer={'weights':tf.Variable(tf.random_normal([len(train_x[0]),n_nodes_h1])),
                    'biases':tf.Variable(tf.random_normal([n_nodes_h1]))}
    hidden_2_layer={'weights':tf.Variable(tf.random_normal([n_nodes_h1,n_nodes_h2])),
                    'biases':tf.Variable(tf.random_normal([n_nodes_h2]))}
    hidden_3_layer={'weights':tf.Variable(tf.random_normal([n_nodes_h2,n_nodes_h3])),
                    'biases':tf.Variable(tf.random_normal([n_nodes_h3]))}
    output_layer={'weights':tf.Variable(tf.random_normal([n_nodes_h3,n_classes])),
                  'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    l1=tf.add(tf.matmul(data,hidden_1_layer['weights']),hidden_1_layer['biases'])
    l1=tf.nn.sigmoid(l1)
    
    l2=tf.add(tf.matmul(l1,hidden_2_layer['weights']),hidden_2_layer['biases'])
    l2=tf.nn.sigmoid(l2)
    
    l3=tf.add(tf.matmul(l2,hidden_3_layer['weights']),hidden_3_layer['biases'])
    l3=tf.nn.sigmoid(l3)
    
    output = tf.matmul(l3,output_layer['weights'])+output_layer['biases']
    output = tf.tanh(output)
    
    #return predicted model.
    return output

In [5]:
#train the model.

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_sum(tf.pow(y - prediction, 2))/(2)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)
    
    #Cross entropy - I tried it using tensforflow. It doesn't worked out.
    #cost = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        hm_epochs = 1
        old_error = 10000000
        correct = tf.equal(tf.round(prediction), y)
        accuracy=tf.reduce_mean(tf.cast(correct,'float'))
        
#         print('Before Training : ')
#         print('Accuracy for training data',accuracy.eval({x:train_x,y:train_y}))
#         print('Accuracy for testing data',accuracy.eval({x:test_x,y:test_y}))
        
        #training starts here : 
        while(True):
            i=0
            error = 0
            while i < len(train_x):
                start = i
                end = i + batch_size
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                k, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
                error += c
                i +=batch_size   
            print('Loss occured for epoch ',hm_epochs,'is :',error)
            hm_epochs = hm_epochs + 1
            if(abs(error-old_error)<=0.00001) or (hm_epochs>=2500):
                break
            else:
                old_error = error
        print('After Training : ')
        print('Accuracy for training data',accuracy.eval({x:train_x,y:train_y}))
        print('Accuracy for testing data',accuracy.eval({x:test_x,y:test_y}))

        prediction = tf.round(prediction)
        outputvalues = prediction.eval(feed_dict={x:test_x,y:test_y})
        #print(outputvalues[0:20] , test_y[0:20])
    
    return outputvalues

In [6]:
inputdata = readfile('SC_DJI_PHANTOM_SET_ONE')
x = []
y = []
for i in inputdata: 
    x.append(i[1:len(i)])
    y.append([i[0]])
x = np.array(x)
#Now x has 90 features with 100000 set of data. Perform PCA and reduce it to 10 features.
matrix_w = doPCA(x)
x = np.array(x.dot(matrix_w))
y = np.array(y)

[93.47083524485069, 3.3647285790189025, 1.8702043138132105, 1.0024019204183938, 0.10194308932451698, 0.086649635527038568, 0.068294072962920394, 0.016054913996866274, 0.0060493414260834174, 0.0032116826837202753]
Max Covariance after dimensionality reduction :  99.990372794


In [7]:
print(x[0])

[ 0.08994633 -0.00739611  0.03509032  0.00732614 -0.05118924 -0.05998804
 -0.02795827 -0.0084756  -0.03756442  0.00149159]


In [8]:
#Now assign this data to train_x and train_y
train_x =list(x[:])
train_y =list(y[:])

In [9]:
print(train_x[0],len(train_x),len(train_x[0])) #'''x[0] => first row input values of 90 features , len(x)=100000, len(x[0]) = 1'''
print(train_y[0],len(train_y))  #'''y[0] => first row output values of 90 features , len(y)=100000, len(y[0]) = 1'''

[ 0.08994633 -0.00739611  0.03509032  0.00732614 -0.05118924 -0.05998804
 -0.02795827 -0.0084756  -0.03756442  0.00149159] 100000 10
[ 0.] 100000


In [10]:
#I am using second file for validation and testing.
testdata = readfile('SC_DJI_PHANTOM_SET_TWO')
testx = []
testy = []
for i in inputdata: 
    testx.append(i[1:len(i)])
    testy.append([i[0]])
testx = np.array(np.array(testx).dot(matrix_w))
testx = np.array(testx[:])
testy = np.array(testy[:])
test_x = list(testx)
test_y = list(testy)

In [11]:
print(test_x[0],len(test_x),len(test_x[0])) #'''x[0] => first row input values of 90 features , len(x)=100000, len(x[0]) = 1'''
print(test_y[0],len(test_y))  #'''y[0] => first row output values of 90 features , len(y)=100000, len(y[0]) = 1'''

[ 0.08994633 -0.00739611  0.03509032  0.00732614 -0.05118924 -0.05998804
 -0.02795827 -0.0084756  -0.03756442  0.00149159] 100000 10
[ 0.] 100000


In [12]:
# x takes the input data of shape None , 10
# here None represents any number of values. 
# 10 represents the features we got after PCA.
x=tf.placeholder('float',[None,len(train_x[0])])
y=tf.placeholder('float',[None,1])

In [13]:
n_nodes_h1 = 10
n_nodes_h2 = 8
n_nodes_h3 = 5
n_classes = 1
batch_size=10000

In [14]:
outputModel = train_neural_network(x)

Loss occured for epoch  1 is : 38296.4992676
Loss occured for epoch  2 is : 31264.015625
Loss occured for epoch  3 is : 30862.8330078
Loss occured for epoch  4 is : 30803.2958984
Loss occured for epoch  5 is : 30782.8251953
Loss occured for epoch  6 is : 30773.7880859
Loss occured for epoch  7 is : 30770.034668
Loss occured for epoch  8 is : 30768.6022949
Loss occured for epoch  9 is : 30768.1599121
Loss occured for epoch  10 is : 30768.0480957
Loss occured for epoch  11 is : 30768.0129395
Loss occured for epoch  12 is : 30768.0266113
Loss occured for epoch  13 is : 30767.9885254
Loss occured for epoch  14 is : 30767.9829102
Loss occured for epoch  15 is : 30767.9804688
Loss occured for epoch  16 is : 30767.9748535
Loss occured for epoch  17 is : 30767.9963379
Loss occured for epoch  18 is : 30768.0063477
Loss occured for epoch  19 is : 30768.0014648
Loss occured for epoch  20 is : 30767.9921875
Loss occured for epoch  21 is : 30767.9628906
Loss occured for epoch  22 is : 30767.967041


In [17]:
print(len(outputModel))

100000


In [18]:
#Question1
ans = []

for i in outputModel:
    if(i==0 or i==-1) :
        ans.append(-1)
    else :
        ans.append(1)
print('First 10 samples of Answer 1 :')
print(ans[0:20])

#Quetion2
ans2 = []
for i in outputModel:
    if(i==0 or i==1) :
        ans2.append(-1)
    else :
        ans2.append(1) 
        
print('First 10 samples of Answer 2 :')
print(ans2[0:20])

[-1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1]
